## Cleaning up

Every time I run the analysis I must paste a hundred lines of code to import files, load network data, etc. Then I can run the analysis and look at results. I need to fold this repetitive process into the body of code itself.

## New data inlet

In [ ]:
# Analysis:
addprocs(4)
@everywhere include("../src/TemporalInstanton.jl")
@everywhere using TemporalInstanton

In [1]:
# Analysis:
include("../src/TemporalInstanton.jl")
using TemporalInstanton

In [3]:
# Far fewer lines of code than previous data loading routine:
Ridx,Y,
Gp,Dp,Rp,
Sb,ref,lines,
res,reac,k,
line_lengths,line_conductors = load_rts96_data();

In [ ]:
# time_intervals = 3 # thirty minutes
# time_values = 0:60:600 # ten minutes in 1-min steps
# int_length = 600. # seconds
# G0 = [0.7*Gp;0.7*Gp;0.7*Gp]
# D0 = [0.9*Dp;0.9*Dp;0.9*Dp]
# P0 = [Rp;1.5*Rp;1.5*Rp]

time_intervals = 6 # thirty minutes total
time_values = 0:30:300 # five minutes in 30-sec steps
int_length = 300. # seconds = 5 min
G0 = [0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp]
D0 = [0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp]
P0 = [Rp;1.1*Rp;1.2*Rp;1.3*Rp;1.4*Rp;1.5*Rp]

# Thermal model parameters:
Tamb = 35. # C
T0 = 60. #46. # initial line steady-state temp

In [ ]:
@time score,x,θ,α,diffs,xopt = solve_temporal_instanton(
    Ridx,Y,G0,P0,D0,Sb,ref,lines,
    res,reac,k,line_lengths,line_conductors,Tamb,
    T0,int_length);

## Temporal instanton analysis ingredients

Temporal instanton analysis requires the following ingredients, regardless of data source:

| Var name        | Description                                  |
|:---------------:|:---------------------------------------------|
| `Ridx`            | Vector: indices of wind nodes                |
| `Y`               | admittance matrix                            |
| `G0`              | Conventional generation                      |
| `P0`              | Renewable forecast                           |
| `D0`              | Demand                                       |
| `Sb`              | System base MVA                              |
| `ref`             | index of angle reference bus                 |
| `lines`           | vector of tuples: (from,to)                  |
| `res`             | pu line resistances                          |
| `reac`            | pu line reactances                           |
| `k`               | generator participation factors              |
| `line_lengths`    | line lengths in meters                       |
| `line_conductors` | vector of strings containing conductor names |
| `Tamb`            | system ambient temperature                   |
| `T0`              | Line initial temperature                     |
| `int_length`      | length of each time interval in seconds      |

I should get rid of `T0` and use forecast information to estimate the line's initial steady-state temperature. Not all lines in the network start out at the same temp. Everything else, however, is pretty much required (though I could alter how the data is packaged).